크롤링

In [1]:
import selenium #셀레니움
import pandas as pd #csv를 읽고 dataframe을 사용하기 위한 pandas
from selenium import webdriver #브라우저를 띄우고 컨트롤하기 위한 webdriver
from selenium.webdriver.common.keys import Keys #브라우저에 키입력 용
from selenium.webdriver.common.by import By #webdriver를 이용해 태그를 찾기 위함
from selenium.webdriver.support.ui import WebDriverWait #Explicitly wait을 위함
from webdriver_manager.chrome import ChromeDriverManager #크롬에서 크롤링 진행 크롬 웹 드라이버 설치시 필요
from selenium.webdriver.support import expected_conditions as EC #브라우저에 특정 요소 상태 확인을 위해
from bs4 import BeautifulSoup #브라우저 태그를 가져오고 파싱하기 위함
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException #예외처리를 위한 예외들
from time import sleep
#크롬 드라이버 최신 버전 https://googlechromelabs.github.io/chrome-for-testing/

c:\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [46]:
## 이 코드는 전에 네이버 API로 한거

df=pd.read_csv("서울 중구 카페.csv",encoding="CP949")
df.drop('Unnamed: 0',axis=1,inplace=True)
#df=df.head(50)
df_no_duplicates = df.drop_duplicates(subset=df.columns)
#주소 간소화시켜주는 함수(ex) 명달로 9길)
def get_addrs(x):
    x1 = x.split(' ')
    return " ".join(x1[2:4])

df_no_duplicates['주소2'] = df['주소'].apply(get_addrs)

import pandas as pd
import requests

#위경도 변환 함수
def get_lat_lng(address):
    client_id = 's8pego1ogd' #각자 네이버 api 받은것
    client_secret = 'IDwfqb34hdFls7BxU0V8WknUzJ0HrQn46NUjfBn9'
    
    url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address}'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret
    }
    
    response = requests.get(url, headers=headers)
    data = response.json()
    
    if 'addresses' in data:
        if len(data['addresses']) > 0:
            latitude = data['addresses'][0]['y']
            longitude = data['addresses'][0]['x']
            return latitude, longitude
        else:
            return None, None
    else:
        return None, None

def convert_address_to_lat_lng(df, address_column):
    latitudes = []
    longitudes = []
    for index, row in df.iterrows():
        address = row[address_column]
        if address:
            latitude, longitude = get_lat_lng(address)
            latitudes.append(latitude)
            longitudes.append(longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
    df['Latitude'] = latitudes
    df['Longitude'] = longitudes


# 주소를 위도와 경도로 변환하여 데이터프레임에 추가
convert_address_to_lat_lng(df_no_duplicates, '주소2')

df_no_duplicates['Latitude'] = df_no_duplicates['Latitude'].astype('float')
df_no_duplicates['Longitude'] = df_no_duplicates['Longitude'].astype('float')

C:\Users\김부겸\AppData\Local\Temp/ipykernel_17132/1708550629.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['주소2'] = df['주소'].apply(get_addrs)
C:\Users\김부겸\AppData\Local\Temp/ipykernel_17132/1708550629.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Latitude'] = latitudes
C:\Users\김부겸\AppData\Local\Temp/ipykernel_17132/1708550629.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [59]:
from folium.plugins import MarkerCluster
from folium import Marker
import folium

lat_m = df_no_duplicates['Latitude'].mean()
long_m = df_no_duplicates['Longitude'].mean()

m = folium.Map(location=[lat_m, long_m], zoom_start=12)

mc = MarkerCluster()

for _, row in df_no_duplicates.iterrows():
    popup_content = f'''
    #클릭했을때의 옵션 조정 계속 글씨가 세로로 써져서 넣어준것임..
    <div style="white-space: nowrap; max-width: 250px;">
    <b>가게명:</b> {row['가게명']}<br>
    <b>업종명:</b> {row['업종']}
    </div>
    '''
    mc.add_child(
        Marker(location=[row['Latitude'], row['Longitude']],
               popup=popup_content
              )
    ).add_to(m)

m
